## Adding features

mel-spec, stft, cqt 피쳐 추가 고려하는버전

In [1]:
import os
import pandas as pd
import numpy as np
import scipy as sp
import scipy.stats
import librosa
import librosa.display
import math
import sys
#sys.path.insert(0,'/home/ikwak2/hmd/notebooks')
#sys.path.insert(0,'/home/ikwak2/hmd/iy_classifier')
sys.path.insert(0,'utils')
from helper_code import *
from get_feature import *
from models import *
from Generator0 import *

In [2]:
root_dir = 'physionet.org/files/circor-heart-sound/1.0.3'
training_data_file = root_dir + '/' + 'training_data.csv'
training_data_dir = root_dir + '/' + 'training_data'
model_dir = root_dir + '/' + 'model'

In [3]:
import tensorflow as tf

gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
print(gpu_info)

Sat Jul  9 17:11:42 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.57.02    Driver Version: 470.57.02    CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  On   | 00000000:00:05.0 Off |                  Off |
| N/A   38C    P0    39W / 300W |      2MiB / 32510MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla V100-SXM2...  On   | 00000000:00:06.0 Off |                  Off |
| N/A   

In [4]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
    except RuntimeError as e:
        print(e)

In [5]:
data_folder =  'physionet.org/files/circor-heart-sound/1.0.3/training_data'
train_folder =  '/home/ubuntu/data/hmd/murmur/train'
test_folder = '/home/ubuntu/data/hmd/murmur/test'


In [6]:
model_folder = 'tmp_model4'

## Modeling



In [7]:
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.callbacks import LearningRateScheduler, ModelCheckpoint

In [8]:
def sigmoidal_decay(e, start=0, end=100, lr_start=1e-3, lr_end=1e-5):
    if e < start:
        return lr_start
    elif e > end:
        return lr_end

    middle = (start + end) / 2
    s = lambda x: 1 / (1 + np.exp(-x))

    return s(13 * (-e + middle) / np.abs(end - start)) * np.abs(lr_start - lr_end) + lr_end

### get feature 함수확장: 음성피쳐 옵션들과, 추가 음성들 고려한 피쳐추출

In [9]:
def get_features_3lb_all(data_folder, patient_files_trn, 
                          samp_sec=20, pre_emphasis = 0, hop_length=256, win_length = 512, n_mels = 100,
                          filter_scale = 1, n_bins = 80, fmin = 10
                         ) :
    features = dict()
    features['id'] = []
    features['age'] = []
    features['sex'] = []
    features['hw'] = []
    features['preg'] = []
    features['loc'] = []
    features['mel1'] = []
    features['cqt1'] = []
    features['stft1'] = []
#    labels = []
    mm_labels = []
    out_labels = []

    age_classes = ['Neonate', 'Infant', 'Child', 'Adolescent', 'Young Adult']
    recording_locations = ['AV', 'MV', 'PV', 'TV', 'PhC']

    num_patient_files = len(patient_files_trn)

    for i in range(num_patient_files):

        # Load the current patient data and recordings.
        current_patient_data = load_patient_data(patient_files_trn[i])
        num_locations = get_num_locations(current_patient_data)
        recording_information = current_patient_data.split('\n')[1:num_locations+1]
        for j in range(num_locations) :
            entries = recording_information[j].split(' ')
            recording_file = entries[2]
            filename = os.path.join(data_folder, recording_file)

            # Extract id
            id1 = recording_file.split('_')[0]
            features['id'].append(id1)

            # Extract melspec
            mel1 = feature_extract_melspec(filename, samp_sec=samp_sec, pre_emphasis = pre_emphasis, hop_length=hop_length, 
                                           win_length = win_length, n_mels = n_mels)[0]
            features['mel1'].append(mel1)
            mel2 = feature_extract_cqt(filename, samp_sec=samp_sec, pre_emphasis = pre_emphasis, filter_scale = filter_scale, 
                                        n_bins = n_bins, fmin = fmin)[0]
            features['cqt1'].append(mel2)
            mel3 = feature_extract_stft(filename, samp_sec=samp_sec, pre_emphasis = pre_emphasis, hop_length=hop_length, 
                                       win_length = win_length)[0]
            features['stft1'].append(mel3)
            
            # Extract age_group
            age_group = get_age(current_patient_data)
            current_age_group = np.zeros(6, dtype=int)
            if age_group in age_classes:
                j = age_classes.index(age_group)
                current_age_group[j] = 1
            else :
                current_age_group[5] = 1
            features['age'].append(current_age_group)

            # Extract sex
            sex = get_sex(current_patient_data)
            sex_features = np.zeros(2, dtype=int)
            if compare_strings(sex, 'Female'):
                sex_features[0] = 1
            elif compare_strings(sex, 'Male'):
                sex_features[1] = 1
            features['sex'].append(sex_features)

            # Extract height and weight.
            height = get_height(current_patient_data)
            weight = get_weight(current_patient_data)
            ## simple impute
            if math.isnan(height) :
                height = 110.846
            if math.isnan(weight) :
                weight = 23.767
                
            features['hw'].append(np.array([height, weight]))

            # Extract pregnancy
            is_pregnant = get_pregnancy_status(current_patient_data)
            features['preg'].append(is_pregnant)

            # Extract location
            locations = entries[0]
            num_recording_locations = len(recording_locations)
            loc_features = np.zeros(num_recording_locations)
            if locations in recording_locations:
                j = recording_locations.index(locations)
                loc_features[j] = 1
            features['loc'].append(loc_features)

            # Extract labels 
            mm_label = get_murmur(current_patient_data)
            out_label = get_outcome(current_patient_data)
            current_mm_labels = np.zeros(2)
            current_out_labels = np.zeros(2)
            if mm_label == 'Absent' :
                current_mm_labels = np.array([0, 0, 1])
            elif mm_label == 'unknown' :
                current_mm_labels = np.array([0, 1, 0])
            else :
                mm_loc = get_murmur_loc(current_patient_data)
                if mm_loc == 'nan' :
                    current_mm_labels = np.array([0.9, 0.05, 0.05])
                else :
                    mm_loc = mm_loc.split('+')
                    if locations in mm_loc :
                        current_mm_labels = np.array([1, 0, 0])
                    else :
                        current_mm_labels = np.array([0.7, 0.2, 0.1])

            if out_label == 'Normal' :
                current_out_labels = np.array([0, 1])
            else :
                current_out_labels = np.array([1, 0])
#                if mm_label == 'Absent' :
#                    current_out_labels = np.array([0.8, 0.2])
#                elif mm_label == 'unknown' :
#                    current_out_labels = np.array([0.85, 0.15])
#                else :
#                    current_out_labels = np.array([1, 0])
                
            mm_labels.append(current_mm_labels)
            out_labels.append(current_out_labels)

    M, N = features['mel1'][i].shape
    for i in range(len(features['mel1'])) :
        features['mel1'][i] = features['mel1'][i].reshape(M,N,1)
    print("melspec: ", M,N)
    mel_input_shape = (M,N,1)
        
    M, N = features['cqt1'][i].shape
    for i in range(len(features['cqt1'])) :
        features['cqt1'][i] = features['cqt1'][i].reshape(M,N,1)
    print("cqt: ", M,N)
    cqt_input_shape = (M,N,1)

    M, N = features['stft1'][i].shape
    for i in range(len(features['stft1'])) :
        features['stft1'][i] = features['stft1'][i].reshape(M,N,1)
    print("stft: ", M,N)
    stft_input_shape = (M,N,1)
        
    for k1 in features.keys() :
        features[k1] = np.array(features[k1])
    
    mm_labels = np.array(mm_labels)
    out_labels = np.array(out_labels)
    return features, mm_labels, out_labels, mel_input_shape, cqt_input_shape, stft_input_shape


In [10]:
patient_files_trn = find_patient_files(train_folder)
patient_files_test = find_patient_files(test_folder)

In [11]:
params_feature = {'samp_sec': 20,
                  #### melspec, stft 피쳐 옵션들  
                  'pre_emphasis': 0,
                  'hop_length': 256,
                  'win_length':512,
                  'n_mels': 100,
                  #### cqt 피쳐 옵션들  
                  'filter_scale': 1,
                  'n_bins': 80,
                  'fmin': 10
}

In [12]:
features_trn, mm_lbs_trn, out_lbs_trn, mel_input_shape, cqt_input_shape, stft_input_shape = get_features_3lb_all(train_folder, patient_files_trn, **params_feature)
features_test, mm_lbs_test, out_lbs_test, _, _, _ = get_features_3lb_all(test_folder, patient_files_test, **params_feature)

melspec:  100 313
cqt:  80 157
stft:  257 313
melspec:  100 313
cqt:  80 157
stft:  257 313


In [13]:
print(mel_input_shape,cqt_input_shape,stft_input_shape)

(100, 313, 1) (80, 157, 1) (257, 313, 1)


### 음성 피쳐 여러개 고려한 모형 함수 짜기

In [14]:
def get_toy5_1(mel_input_shape, cqt_input_shape, stft_input_shape, use_mel = True, use_cqt = True, use_stft = True):
        # Create a towy model.
    age = keras.Input(shape=(6,), name = 'age_cat')
    sex = keras.Input(shape=(2,), name = 'sex_cat')
    hw = keras.Input(shape=(2,), name = 'height_weight')
    preg = keras.Input(shape=(1,), name = 'is_preg')
    loc = keras.Input(shape=(5,), name = 'loc')
    mel1 = keras.Input(shape=(mel_input_shape), name = 'mel')
    cqt1 = keras.Input(shape=(cqt_input_shape), name = 'cqt')
    stft1 = keras.Input(shape=(stft_input_shape), name = 'stft')
        
        
    ## age embeddig
    age1 = layers.Dense(2, activation = None)(age)

    ## sex embedding
    sex1 = layers.Dense(1, activation = None)(sex)

    ## hw embedding
    hw1 = layers.Dense(1, activation = None)(hw)

    ## loc embedding
    loc1 = layers.Dense(3, activation = None)(loc)

    ## mel embedding
    mel2 = layers.Conv2D(16, (3,3), activation = 'relu')(mel1)
    mel2 = layers.MaxPooling2D()(mel2)
    mel2 = layers.Conv2D(32, (5,5), activation = 'relu')(mel2)
    mel2 = layers.MaxPooling2D()(mel2)
    mel2 = layers.Conv2D(32, (3,3), activation = 'relu')(mel2)
    mel2 = layers.MaxPooling2D()(mel2)
    mel2 = layers.Conv2D(64, (3,3), activation = 'relu')(mel2)
    mel2 = layers.MaxPooling2D()(mel2)
    mel2 = layers.GlobalAveragePooling2D()(mel2)

    ## cqt embedding
    cqt2 = layers.Conv2D(16, (3,3), activation = 'relu')(cqt1)
    cqt2 = layers.MaxPooling2D()(cqt2)
    cqt2 = layers.Conv2D(32, (5,5), activation = 'relu')(cqt2)
    cqt2 = layers.MaxPooling2D()(cqt2)
    cqt2 = layers.Conv2D(32, (3,3), activation = 'relu')(cqt2)
    cqt2 = layers.MaxPooling2D()(cqt2)
    cqt2 = layers.Conv2D(64, (3,3), activation = 'relu')(cqt2)
    cqt2 = layers.MaxPooling2D()(cqt2)
    cqt2 = layers.GlobalAveragePooling2D()(cqt2)

    ## stft embedding
    stft2 = layers.Conv2D(16, (3,3), activation = 'relu')(stft1)
    stft2 = layers.MaxPooling2D()(stft2)
    stft2 = layers.Conv2D(32, (5,5), activation = 'relu')(stft2)
    stft2 = layers.MaxPooling2D()(stft2)
    stft2 = layers.Conv2D(32, (3,3), activation = 'relu')(stft2)
    stft2 = layers.MaxPooling2D()(stft2)
    stft2 = layers.Conv2D(64, (3,3), activation = 'relu')(stft2)
    stft2 = layers.MaxPooling2D()(stft2)
    stft2 = layers.GlobalAveragePooling2D()(stft2)
    
#    concat1 = layers.Concatenate()([age1, sex1, hw1, loc1, preg])
#    d1 = layers.Dense(2, activation = 'relu')(concat1)
    
    if use_mel and use_cqt and use_stft :
        concat2 = layers.Concatenate()([mel2, cqt2, stft2])
    if not use_mel and use_cqt and use_stft :
        concat2 = layers.Concatenate()([cqt2, stft2])
    if use_mel and not use_cqt and use_stft :
        concat2 = layers.Concatenate()([mel2, stft2])
    if use_mel and use_cqt and not use_stft :
        concat2 = layers.Concatenate()([mel2, cqt2])
    if not use_mel and not use_cqt and use_stft :  ## stft 만
        concat2 = layers.Concatenate()([stft2])
    if use_mel and not use_cqt and not use_stft :  ### mel만
        concat2 = layers.Concatenate()([mel2])
    if not use_mel and use_cqt and not use_stft :  ### cqt만
        concat2 = layers.Concatenate()([cqt2])
#    concat2 = layers.Dense(10, activation = 'relu')(concat2)
    res1 = layers.Dense(3, activation = "softmax")(concat2)
    res2 = layers.Dense(2, activation = "softmax")(concat2)

    model = keras.Model(inputs = [age,sex,hw,preg,loc,mel1,cqt1,stft1] , outputs = res1 )
    
    model.compile(loss=['categorical_crossentropy'], optimizer='adam', metrics=['accuracy'])
    return(model)

def get_toy5_2(mel_input_shape, cqt_input_shape, stft_input_shape, use_mel = True, use_cqt = True, use_stft = True):
        # Create a towy model.
    age = keras.Input(shape=(6,), name = 'age_cat')
    sex = keras.Input(shape=(2,), name = 'sex_cat')
    hw = keras.Input(shape=(2,), name = 'height_weight')
    preg = keras.Input(shape=(1,), name = 'is_preg')
    loc = keras.Input(shape=(5,), name = 'loc')
    mel1 = keras.Input(shape=(mel_input_shape), name = 'mel')
    cqt1 = keras.Input(shape=(cqt_input_shape), name = 'cqt')
    stft1 = keras.Input(shape=(stft_input_shape), name = 'stft')
        
        
    ## age embeddig
    age1 = layers.Dense(2, activation = None)(age)

    ## sex embedding
    sex1 = layers.Dense(1, activation = None)(sex)

    ## hw embedding
    hw1 = layers.Dense(1, activation = None)(hw)

    ## loc embedding
    loc1 = layers.Dense(3, activation = None)(loc)

    ## mel embedding
    mel2 = layers.Conv2D(16, (3,3), activation = 'relu')(mel1)
    mel2 = layers.MaxPooling2D()(mel2)
    mel2 = layers.Conv2D(32, (5,5), activation = 'relu')(mel2)
    mel2 = layers.MaxPooling2D()(mel2)
    mel2 = layers.Conv2D(32, (3,3), activation = 'relu')(mel2)
    mel2 = layers.MaxPooling2D()(mel2)
    mel2 = layers.Conv2D(64, (3,3), activation = 'relu')(mel2)
    mel2 = layers.MaxPooling2D()(mel2)
    mel2 = layers.GlobalAveragePooling2D()(mel2)

    ## cqt embedding
    cqt2 = layers.Conv2D(16, (3,3), activation = 'relu')(cqt1)
    cqt2 = layers.MaxPooling2D()(cqt2)
    cqt2 = layers.Conv2D(32, (5,5), activation = 'relu')(cqt2)
    cqt2 = layers.MaxPooling2D()(cqt2)
    cqt2 = layers.Conv2D(32, (3,3), activation = 'relu')(cqt2)
    cqt2 = layers.MaxPooling2D()(cqt2)
    cqt2 = layers.Conv2D(64, (3,3), activation = 'relu')(cqt2)
    cqt2 = layers.MaxPooling2D()(cqt2)
    cqt2 = layers.GlobalAveragePooling2D()(cqt2)

    ## stft embedding
    stft2 = layers.Conv2D(16, (3,3), activation = 'relu')(stft1)
    stft2 = layers.MaxPooling2D()(stft2)
    stft2 = layers.Conv2D(32, (5,5), activation = 'relu')(stft2)
    stft2 = layers.MaxPooling2D()(stft2)
    stft2 = layers.Conv2D(32, (3,3), activation = 'relu')(stft2)
    stft2 = layers.MaxPooling2D()(stft2)
    stft2 = layers.Conv2D(64, (3,3), activation = 'relu')(stft2)
    stft2 = layers.MaxPooling2D()(stft2)
    stft2 = layers.GlobalAveragePooling2D()(stft2)
    
#    concat1 = layers.Concatenate()([age1, sex1, hw1, loc1, preg])
#    d1 = layers.Dense(2, activation = 'relu')(concat1)
    
    if use_mel and use_cqt and use_stft :
        concat2 = layers.Concatenate()([mel2, cqt2, stft2])
    if not use_mel and use_cqt and use_stft :
        concat2 = layers.Concatenate()([cqt2, stft2])
    if use_mel and not use_cqt and use_stft :
        concat2 = layers.Concatenate()([mel2, stft2])
    if use_mel and use_cqt and not use_stft :
        concat2 = layers.Concatenate()([mel2, cqt2])
    if not use_mel and not use_cqt and use_stft :  ## stft만
        concat2 = layers.Concatenate()([stft2])
    if use_mel and not use_cqt and not use_stft :  ### mel만
        concat2 = layers.Concatenate()([mel2])
    if not use_mel and use_cqt and not use_stft :  ### cqt만
        concat2 = layers.Concatenate()([cqt2])
#    concat2 = layers.Dense(10, activation = 'relu')(concat2)
    res1 = layers.Dense(3, activation = "softmax")(concat2)
    res2 = layers.Dense(2, activation = "softmax")(concat2)

    model = keras.Model(inputs = [age,sex,hw,preg,loc,mel1,cqt1,stft1] , outputs = res2 )
    
    model.compile(loss=['categorical_crossentropy'], optimizer='adam', metrics=['accuracy'])
    return(model)

In [15]:
model1 = get_toy5_1(mel_input_shape, cqt_input_shape, stft_input_shape)
model2 = get_toy5_2(mel_input_shape, cqt_input_shape, stft_input_shape)


epoch2로 돌아가는지 정도만 확인

In [52]:
n_epoch = 100
lr = LearningRateScheduler(lambda e: sigmoidal_decay(e, end=n_epoch))
batch_size = 64
params = {'batch_size': batch_size,
#          'input_shape': (100, 313, 1),
          'shuffle': True,
          'beta_param': 0.7,
          'mixup': True,
#          'lowpass': [.5, [11,12,13,14,15,16,17,18]]
#          'highpass': [.5, [78,79,80,81,82,83,84,85]]
          'ranfilter2' : [3, [18,19,20,21,22,23]]
#           'dropblock' : [30, 100]
          #'device' : device
}

params_no_shuffle = {'batch_size': batch_size,
#          'input_shape': (100, 313, 1),
          'shuffle': False,
          'beta_param': 0.7,
          'mixup': False
          #'device': device
}

TrainDGen_1 = Generator0([features_trn['age'],features_trn['sex'], features_trn['hw'], features_trn['preg'], features_trn['loc'], 
           features_trn['mel1'],features_trn['cqt1'],features_trn['stft1']], 
                        mm_lbs_trn,  ## our Y
                        **params)()

#ValDGen_1 = Generator0([features_test[0]['age'],features_test[0]['sex'], features_test[0]['hw'], features_test[0]['preg'], features_test[0]['loc'], 
#           features_test[0]['mel1'],features_test[0]['cqt1'],features_test[0]['stft1']], 
#                        mm_lbs_test,  ## our Y
#                        **params_no_shuffle)()

    
model1.fit(TrainDGen_1,
          validation_data = ([features_test['age'],features_test['sex'], features_test['hw'], 
                              features_test['preg'], features_test['loc'], features_test['mel1'], 
                              features_test['cqt1'], features_test['stft1']], 
                             mm_lbs_test), 
                             callbacks=[lr],
                              steps_per_epoch=np.ceil(len(mm_lbs_trn)/64),
                             epochs = n_epoch)

Epoch 1/100
40/40 [==============================] - 8s 197ms/step - loss: 0.5800 - accuracy: 0.7625 - val_loss: 0.5754 - val_accuracy: 0.7559
Epoch 2/100
40/40 [==============================] - 7s 179ms/step - loss: 0.5825 - accuracy: 0.7684 - val_loss: 0.5793 - val_accuracy: 0.7528
Epoch 3/100
40/40 [==============================] - 7s 185ms/step - loss: 0.5722 - accuracy: 0.7672 - val_loss: 0.5777 - val_accuracy: 0.7353
Epoch 4/100
40/40 [==============================] - 7s 185ms/step - loss: 0.5617 - accuracy: 0.7773 - val_loss: 0.5566 - val_accuracy: 0.7591
Epoch 5/100
40/40 [==============================] - 7s 182ms/step - loss: 0.5569 - accuracy: 0.7793 - val_loss: 0.5623 - val_accuracy: 0.7591
Epoch 6/100
40/40 [==============================] - 7s 180ms/step - loss: 0.5369 - accuracy: 0.7973 - val_loss: 0.5613 - val_accuracy: 0.7734
Epoch 7/100
40/40 [==============================] - 7s 175ms/step - loss: 0.5366 - accuracy: 0.7988 - val_loss: 0.5447 - val_accuracy: 0.7750

Epoch 58/100
40/40 [==============================] - 7s 180ms/step - loss: 0.4546 - accuracy: 0.8480 - val_loss: 0.4257 - val_accuracy: 0.8399
Epoch 59/100
40/40 [==============================] - 7s 174ms/step - loss: 0.4677 - accuracy: 0.8340 - val_loss: 0.4232 - val_accuracy: 0.8415
Epoch 60/100
40/40 [==============================] - 7s 184ms/step - loss: 0.4637 - accuracy: 0.8441 - val_loss: 0.4217 - val_accuracy: 0.8494
Epoch 61/100
40/40 [==============================] - 7s 178ms/step - loss: 0.4564 - accuracy: 0.8387 - val_loss: 0.4204 - val_accuracy: 0.8431
Epoch 62/100
40/40 [==============================] - 7s 178ms/step - loss: 0.4520 - accuracy: 0.8508 - val_loss: 0.4377 - val_accuracy: 0.8463
Epoch 63/100
40/40 [==============================] - 7s 176ms/step - loss: 0.4517 - accuracy: 0.8465 - val_loss: 0.4320 - val_accuracy: 0.8463
Epoch 64/100
40/40 [==============================] - 7s 173ms/step - loss: 0.4603 - accuracy: 0.8398 - val_loss: 0.4354 - val_accuracy:

epoch2로 돌아가는지 정도만 확인

In [53]:
n_epoch = 100
lr = LearningRateScheduler(lambda e: sigmoidal_decay(e, end=n_epoch))
batch_size = 64
params = {'batch_size': batch_size,
#          'input_shape': (100, 313, 1),
          'shuffle': True,
          'beta_param': 0.7,
          'mixup': True,
#          'lowpass': [.5, [11,12,13,14,15,16,17,18]]
          'highpass': [.5, [78,79,80,81,82,83,84,85]],
          'ranfilter2' : [3, [18,19,20,21,22,23]]
#           'dropblock' : [30, 100]
          #'device' : device
}

params_no_shuffle = {'batch_size': batch_size,
#          'input_shape': (100, 313, 1),
          'shuffle': False,
          'beta_param': 0.7,
          'mixup': False
          #'device': device
}

TrainDGen_1 = Generator0([features_trn['age'],features_trn['sex'], features_trn['hw'], features_trn['preg'], features_trn['loc'], 
           features_trn['mel1'],features_trn['cqt1'],features_trn['stft1']], 
                        out_lbs_trn,  ## our Y
                        **params)()

#ValDGen_1 = Generator0([features_test[0]['age'],features_test[0]['sex'], features_test[0]['hw'], features_test[0]['preg'], features_test[0]['loc'], 
#           features_test[0]['mel1'],features_test[0]['cqt1'],features_test[0]['stft1']], 
#                        features_test[2],  ## our Y
#                        **params_no_shuffle)()

    
model2.fit(TrainDGen_1,
          validation_data = ([features_test['age'],features_test['sex'], features_test['hw'], 
                              features_test['preg'], features_test['loc'], features_test['mel1'], 
                              features_test['cqt1'], features_test['stft1']], 
                             out_lbs_test), 
                             callbacks=[lr],
                              steps_per_epoch=np.ceil(len(out_lbs_trn)/64),
                             epochs = n_epoch)

Epoch 1/100
40/40 [==============================] - 8s 196ms/step - loss: 0.6835 - accuracy: 0.5566 - val_loss: 0.6725 - val_accuracy: 0.5943
Epoch 2/100
40/40 [==============================] - 7s 186ms/step - loss: 0.6797 - accuracy: 0.5590 - val_loss: 0.6711 - val_accuracy: 0.5959
Epoch 3/100
40/40 [==============================] - 7s 186ms/step - loss: 0.6769 - accuracy: 0.5652 - val_loss: 0.6793 - val_accuracy: 0.5784
Epoch 4/100
40/40 [==============================] - 7s 185ms/step - loss: 0.6795 - accuracy: 0.5648 - val_loss: 0.6998 - val_accuracy: 0.5784
Epoch 5/100
40/40 [==============================] - 7s 191ms/step - loss: 0.6756 - accuracy: 0.5730 - val_loss: 0.6707 - val_accuracy: 0.5975
Epoch 6/100
40/40 [==============================] - 7s 187ms/step - loss: 0.6657 - accuracy: 0.6027 - val_loss: 0.7807 - val_accuracy: 0.5594
Epoch 7/100
40/40 [==============================] - 8s 193ms/step - loss: 0.6665 - accuracy: 0.6156 - val_loss: 0.7036 - val_accuracy: 0.5959

Epoch 58/100
40/40 [==============================] - 7s 188ms/step - loss: 0.6203 - accuracy: 0.6719 - val_loss: 0.7039 - val_accuracy: 0.6450
Epoch 59/100
40/40 [==============================] - 7s 188ms/step - loss: 0.6240 - accuracy: 0.6699 - val_loss: 0.6646 - val_accuracy: 0.6704
Epoch 60/100
40/40 [==============================] - 7s 188ms/step - loss: 0.6244 - accuracy: 0.6621 - val_loss: 0.7368 - val_accuracy: 0.6260
Epoch 61/100
40/40 [==============================] - 7s 178ms/step - loss: 0.6124 - accuracy: 0.6824 - val_loss: 0.6956 - val_accuracy: 0.6735
Epoch 62/100
40/40 [==============================] - 8s 195ms/step - loss: 0.6155 - accuracy: 0.6785 - val_loss: 0.7408 - val_accuracy: 0.6355
Epoch 63/100
40/40 [==============================] - 8s 209ms/step - loss: 0.6124 - accuracy: 0.6789 - val_loss: 0.7801 - val_accuracy: 0.6244
Epoch 64/100
40/40 [==============================] - 8s 208ms/step - loss: 0.6184 - accuracy: 0.6750 - val_loss: 0.8096 - val_accuracy:

### 팀코드 수정해야 할 함수들..

In [55]:
params_feature

{'samp_sec': 20,
 'pre_emphasis': 0,
 'hop_length': 256,
 'win_length': 512,
 'n_mels': 100,
 'filter_scale': 1,
 'n_bins': 80,
 'fmin': 10,
 'mel_shape': (100, 313, 1),
 'cqt_shape': (80, 157, 1),
 'stft_shape': (257, 313, 1),
 'model1': 'toy1',
 'model2': 'toy2',
 'model_fnm1': 'tmp_model4/toy1_model1.hdf5',
 'model_fnm2': 'tmp_model4/toy2_model2.hdf5'}

In [56]:
params_feature['mel_shape'] = mel_input_shape
params_feature['cqt_shape'] = cqt_input_shape
params_feature['stft_shape'] = stft_input_shape


In [57]:
import pickle as pk
def save_challenge_model(model_folder, model1, model2, m_name1, m_name2, param_feature) :
    os.makedirs(model_folder, exist_ok=True)
    info_fnm = os.path.join(model_folder, 'desc.pk')
    filename1 = os.path.join(model_folder, m_name1 + '_model1.hdf5')
    filename2 = os.path.join(model_folder, m_name2 + '_model2.hdf5')
    model1.save(filename1)
    model2.save(filename2)
    param_feature['model1'] = m_name1
    param_feature['model2'] = m_name2
    param_feature['model_fnm1'] = filename1
    param_feature['model_fnm2'] = filename2

    with open(info_fnm, 'wb') as f:
        pk.dump(param_feature, f, pk.HIGHEST_PROTOCOL)
    return 1
    

In [58]:
import pickle as pk
def save_challenge_model(model_folder, model1, model2, m_name1, m_name2, mel_shape = (100, 313, 1)) :
    os.makedirs(model_folder, exist_ok=True)
    info_fnm = os.path.join(model_folder, 'desc.pk')
    filename1 = os.path.join(model_folder, m_name1 + '_model1.hdf5')
    filename2 = os.path.join(model_folder, m_name2 + '_model2.hdf5')
    model1.save(filename1)
    model2.save(filename2)
    d = {'model1': m_name1, 'model2': m_name2, 'mel_shape': mel_shape, 'model_fnm1': filename1, 'model_fnm2': filename2}    
    with open(info_fnm, 'wb') as f:
        pk.dump(d, f, pk.HIGHEST_PROTOCOL)
    return 1
    

In [59]:
model_folder

'tmp_model4'

In [60]:
save_challenge_model(model_folder, model1, model2, m_name1 = 'toy1', m_name2 = 'toy2', param_feature = params_feature)

TypeError: save_challenge_model() got an unexpected keyword argument 'param_feature'

In [35]:
def load_challenge_model(model_folder, verbose):
    info_fnm = os.path.join(model_folder, 'desc.pk')
    with open(info_fnm, 'rb') as f:
        info_m = pk.load(f)
#    if info_m['model'] == 'toy' :
#        model = get_toy(info_m['mel_shape'])
#    filename = os.path.join(model_folder, info_m['model'] + '_model.hdf5')
#    model.load_weights(filename)
    return info_m

In [28]:
params_feature

{'samp_sec': 20,
 'pre_emphasis': 0,
 'hop_length': 256,
 'win_length': 512,
 'n_mels': 100,
 'filter_scale': 1,
 'n_bins': 80,
 'fmin': 10,
 'mel_shape': (100, 313, 1),
 'cqt_shape': (80, 157, 1),
 'stft_shape': (257, 313, 1),
 'model1': 'toy1',
 'model2': 'toy2',
 'model_fnm1': 'tmp_model4/toy1_model1.hdf5',
 'model_fnm2': 'tmp_model4/toy2_model2.hdf5'}

In [ ]:
            # Extract melspec
            mel1 = feature_extract_melspec(filename, samp_sec=samp_sec, pre_emphasis = pre_emphasis, hop_length=hop_length, 
                                           win_length = win_length, n_mels = n_mels)[0]
            features['mel1'].append(mel1)
            mel2 = feature_extract_cqt(filename, samp_sec=samp_sec, pre_emphasis = pre_emphasis, filter_scale = filter_scale, 
                                        n_bins = n_bins, fmin = fmin)[0]
            features['cqt1'].append(mel2)
            mel3 = feature_extract_stft(filename, samp_sec=samp_sec, pre_emphasis = pre_emphasis, hop_length=hop_length, 
                                       win_length = win_length)[0]
            features['stft1'].append(mel3)


In [45]:
def run_challenge_model(model, data, recordings, verbose):
    
    murmur_classes = ['Present', 'Unknown', 'Absent']
    outcome_classes = ['Abnormal', 'Normal']
    
    if model['model1'] == 'toy1' :
        model1 = get_toy5_1(model['mel_shape'],model['cqt_shape'],model['stft_shape'] )
    if model['model2'] == 'toy2' :
        model2 = get_toy5_2(model['mel_shape'],model['cqt_shape'],model['stft_shape'])
    model1.load_weights(model['model_fnm1'])
    model2.load_weights(model['model_fnm2'])
    
#    classes = model['classes']
    # Load features.
    features = get_feature_one(data, verbose = 0)

    samp_sec = model['samp_sec'] 
    pre_emphasis = model['pre_emphasis']
    hop_length = model['hop_length']
    win_length = model['win_length']
    n_mels = model['n_mels']
    filter_scale = model['filter_scale']
    n_bins = model['n_bins']
    fmin = model['fmin']
    
    features['mel1'] = []
    for i in range(len(recordings)) :
        mel1 = feature_extract_melspec(recordings[i]/ 32768, samp_sec=samp_sec, pre_emphasis = pre_emphasis, hop_length=hop_length, 
                                           win_length = win_length, n_mels = n_mels)[0]
        features['mel1'].append(mel1)
    M, N = features['mel1'][0].shape
    for i in range(len(features['mel1'])) :
        features['mel1'][i] = features['mel1'][i].reshape(M,N,1)   
    features['mel1'] = np.array(features['mel1'])

    features['cqt1'] = []
    for i in range(len(recordings)) :
        mel1 = feature_extract_cqt(recordings[i]/ 32768, samp_sec=samp_sec, pre_emphasis = pre_emphasis, filter_scale = filter_scale, 
                                        n_bins = n_bins, fmin = fmin)[0]
        features['cqt1'].append(mel1)
    M, N = features['cqt1'][0].shape
    for i in range(len(features['cqt1'])) :
        features['cqt1'][i] = features['cqt1'][i].reshape(M,N,1)   
    features['cqt1'] = np.array(features['cqt1'])

    features['stft1'] = []
    for i in range(len(recordings)) :
        mel1 = feature_extract_stft(recordings[i]/ 32768, samp_sec=samp_sec, pre_emphasis = pre_emphasis, hop_length=hop_length, 
                                       win_length = win_length)[0]
        features['stft1'].append(mel1)
    M, N = features['stft1'][0].shape
    for i in range(len(features['stft1'])) :
        features['stft1'][i] = features['stft1'][i].reshape(M,N,1)           
    features['stft1'] = np.array(features['stft1'])

    #    print(features)
    # Impute missing data.
    res1 = model1.predict([features['age'], features['sex'], features['hw'], features['preg'], features['loc'], features['mel1'], features['cqt1'], features['stft1']])
    res2 = model2.predict([features['age'], features['sex'], features['hw'], features['preg'], features['loc'], features['mel1'], features['cqt1'], features['stft1']])

    # Get classifier probabilities.
    idx1 = res1.argmax(axis=0)[0]
    murmur_probabilities = res1[idx1,]  ## mumur 확률 최대화 되는 애 뽑기
    outcome_probabilities = res2.mean(axis = 0) ##  outcome 은 그냥 평균으로 뽑기
#    idx = np.argmax(prob1)

    ## 이부분도 생각 필요.. rule 을 cost를 maximize 하는 기준으로 threshold 탐색 필요할지도..
    # Choose label with highest probability.
    murmur_labels = np.zeros(len(murmur_classes), dtype=np.int_)
    idx = np.argmax(murmur_probabilities)
    murmur_labels[idx] = 1
    outcome_labels = np.zeros(len(outcome_classes), dtype=np.int_)
    idx = np.argmax(outcome_probabilities)
    outcome_labels[idx] = 1
    
    # Concatenate classes, labels, and probabilities.
    classes = murmur_classes + outcome_classes
    labels = np.concatenate((murmur_labels, outcome_labels))
    probabilities = np.concatenate((murmur_probabilities, outcome_probabilities))
    
    return classes, labels, probabilities

### 대회 평가용 run_model 함수 (수정 불필요)

In [46]:
# Run model.
def run_model(model_folder, data_folder, output_folder, allow_failures, verbose):
    # Load models.
    if verbose >= 1:
        print('Loading Challenge model...')

    model = load_challenge_model(model_folder, verbose) ### Teams: Implement this function!!!

    # Find the patient data files.
    patient_files = find_patient_files(data_folder)
    num_patient_files = len(patient_files)

    if num_patient_files==0:
        raise Exception('No data was provided.')

    # Create a folder for the Challenge outputs if it does not already exist.
    os.makedirs(output_folder, exist_ok=True)

    # Run the team's model on the Challenge data.
    if verbose >= 1:
        print('Running model on Challenge data...')

    # Iterate over the patient files.
    for i in range(num_patient_files):
        if verbose >= 2:
            print('    {}/{}...'.format(i+1, num_patient_files))

        patient_data = load_patient_data(patient_files[i])
        recordings = load_recordings(data_folder, patient_data)

        # Allow or disallow the model to fail on parts of the data; helpful for debugging.
        try:
            classes, labels, probabilities = run_challenge_model(model, patient_data, recordings, verbose) ### Teams: Implement this function!!!
        except:
            if allow_failures:
                if verbose >= 2:
                    print('... failed.')
                classes, labels, probabilities = list(), list(), list()
            else:
                raise

        # Save Challenge outputs.
        head, tail = os.path.split(patient_files[i])
        root, extension = os.path.splitext(tail)
        output_file = os.path.join(output_folder, root + '.csv')
        patient_id = get_patient_id(patient_data)
        save_challenge_outputs(output_file, patient_id, classes, labels, probabilities)

    if verbose >= 1:
        print('Done.')

In [31]:
## 우리 모형 저장된 폴더이름
model_folder

'tmp_model4'

In [32]:
## 테스트 데이터 폴더
test_folder

'/home/ubuntu/data/hmd/murmur/test'

In [33]:
## 테스트 데이터에 모형 돌려서 스코어 저장할 폴더
output_folder = '/home/ubuntu/hmd/notebooks/out2'

In [48]:
run_model(model_folder, test_folder, output_folder, allow_failures = True, verbose = 1)

Loading Challenge model...
Running model on Challenge data...


Done.


In [49]:
!cat /home/ubuntu/hmd/notebooks/out2/49628.csv

#49628
Present,Unknown,Absent,Abnormal,Normal
0,0,1,1,0
0.32305384,0.02188414,0.655062,0.61604756,0.38395247


In [50]:
from evaluate_model import *

In [51]:
murmur_scores, outcome_scores = evaluate_model(test_folder, output_folder)
classes, auroc, auprc, auroc_classes, auprc_classes, f_measure, f_measure_classes, accuracy, accuracy_classes, weighted_accuracy, cost = murmur_scores
murmur_output_string = 'AUROC,AUPRC,F-measure,Accuracy,Weighted Accuracy,Cost\n{:.3f},{:.3f},{:.3f},{:.3f},{:.3f},{:.3f}\n'.format(auroc, auprc, f_measure, accuracy, weighted_accuracy, cost)
murmur_class_output_string = 'Classes,{}\nAUROC,{}\nAUPRC,{}\nF-measure,{}\nAccuracy,{}\n'.format(
    ','.join(classes),
    ','.join('{:.3f}'.format(x) for x in auroc_classes),
    ','.join('{:.3f}'.format(x) for x in auprc_classes),
    ','.join('{:.3f}'.format(x) for x in f_measure_classes),
    ','.join('{:.3f}'.format(x) for x in accuracy_classes))

classes, auroc, auprc, auroc_classes, auprc_classes, f_measure, f_measure_classes, accuracy, accuracy_classes, weighted_accuracy, cost = outcome_scores
outcome_output_string = 'AUROC,AUPRC,F-measure,Accuracy,Weighted Accuracy,Cost\n{:.3f},{:.3f},{:.3f},{:.3f},{:.3f},{:.3f}\n'.format(auroc, auprc, f_measure, accuracy, weighted_accuracy, cost)
outcome_class_output_string = 'Classes,{}\nAUROC,{}\nAUPRC,{}\nF-measure,{}\nAccuracy,{}\n'.format(
    ','.join(classes),
    ','.join('{:.3f}'.format(x) for x in auroc_classes),
    ','.join('{:.3f}'.format(x) for x in auprc_classes),
    ','.join('{:.3f}'.format(x) for x in f_measure_classes),
    ','.join('{:.3f}'.format(x) for x in accuracy_classes))

output_string = '#Murmur scores\n' + murmur_output_string + '\n#Outcome scores\n' + outcome_output_string \
    + '\n#Murmur scores (per class)\n' + murmur_class_output_string + '\n#Outcome scores (per class)\n' + outcome_class_output_string

if len(sys.argv) == 3:
    print(output_string)
elif len(sys.argv) == 4:
    with open(sys.argv[3], 'w') as f:
        f.write(output_string)

#Murmur scores
AUROC,AUPRC,F-measure,Accuracy,Weighted Accuracy,Cost
0.688,0.435,0.281,0.728,0.375,25689.450

#Outcome scores
AUROC,AUPRC,F-measure,Accuracy,Weighted Accuracy,Cost
0.702,0.693,0.459,0.545,0.274,22779.947

#Murmur scores (per class)
Classes,Present,Unknown,Absent
AUROC,0.631,0.738,0.695
AUPRC,0.273,0.171,0.860
F-measure,0.000,0.000,0.842
Accuracy,0.000,0.000,1.000

#Outcome scores (per class)
Classes,Abnormal,Normal
AUROC,0.702,0.702
AUPRC,0.703,0.682
F-measure,0.243,0.674
Accuracy,0.143,0.968



epoch 2 트레이닝 해서 결과는 별로.. 기록은 잘 됨